In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import datetime
import matplotlib.pyplot as plt

sys.path.insert(0, r"/mnt/c/Users/Trez/Desktop/tudat-bundle/tudatpy/")

# Tudat imports for propagation and estimation
from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation, constants
from tudatpy.kernel.numerical_simulation import environment_setup
from tudatpy.kernel.numerical_simulation import propagation_setup
from tudatpy.kernel.numerical_simulation import estimation, estimation_setup
from tudatpy.kernel.numerical_simulation.estimation_setup import observation

# import MPC interface
from tudatpy.data.mpc import BatchMPC
from tudatpy.data.horizons import HorizonsQuery, HorizonsBatch

# other useful modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

from astropy.time import Time

In [2]:
batch = BatchMPC()
# batch = batch.filter(observatories=["089"])
batch.get_observations([433])
batch.filter(
    epoch_start=datetime.datetime(2021, 11, 5),
    epoch_end=datetime.datetime(2022, 1, 1),
    observatories=["089"],
)
batch_times = batch.table.epochJ2000secondsTDB.values

print(batch_times)
print(batch.size)

[6.89748536e+08 6.89748734e+08 6.89749130e+08 6.89749329e+08
 6.89830133e+08 6.89830733e+08 6.89831333e+08 6.89831930e+08
 6.89832528e+08 6.89833128e+08 6.89833729e+08 6.89834328e+08
 6.89834928e+08 6.89835527e+08]
14


In [3]:
venus = HorizonsQuery(
    query_id="299",
    location="@SSB",
    epoch_start=datetime.datetime(2021, 1, 1),
    epoch_end=datetime.datetime(2023, 1, 1),
    epoch_step="1d",
)
eros = HorizonsQuery(
    query_id="433;",
    location="089@399",
    epoch_list=batch_times
)
eros2 = HorizonsQuery(
    query_id="433;",
    location="089@399",
    epoch_list=batch_times
)

In [4]:
a = venus.ephemerides()

print(a)

 targetname       datetime_str       ...  PABLat epochJ2000secondsTDB
    ---                              ...   deg                       
----------- ------------------------ ... ------- --------------------
Venus (299) 2021-Jan-01 00:00:00.000 ...  1.4249          662731200.0
Venus (299) 2021-Jan-02 00:00:00.000 ...  1.3385          662817600.0
Venus (299) 2021-Jan-03 00:00:00.000 ...  1.2512          662904000.0
Venus (299) 2021-Jan-04 00:00:00.000 ...  1.1629          662990400.0
Venus (299) 2021-Jan-05 00:00:00.000 ...  1.0737          663076800.0
Venus (299) 2021-Jan-06 00:00:00.000 ...  0.9837          663163200.0
Venus (299) 2021-Jan-07 00:00:00.000 ...  0.8929          663249600.0
Venus (299) 2021-Jan-08 00:00:00.000 ...  0.8014          663336000.0
Venus (299) 2021-Jan-09 00:00:00.000 ...  0.7093          663422400.0
Venus (299) 2021-Jan-10 00:00:00.000 ...  0.6167          663508800.0
        ...                      ... ...     ...                  ...
Venus (299) 2022-Dec

In [5]:
epoch = {
    "start": "2021-01-01 00:00:00.000000",
    "stop": "2023-01-01 00:00:00.000000",
    "step": "1d",
}


start = Time(epoch["start"], format="iso", scale="tdb").utc
# start.format = 'tdb'

print(start)

start = start.utc

print(start)

2020-12-31 23:58:50.816
2020-12-31 23:58:50.816


### RADEC comparison

In [6]:
radec_mpc = batch.table.loc[:, ["epochJ2000secondsTDB", "RA", "DEC"]].reset_index(
    drop=True
)
radec_horizons = (
    eros.ephemerides(extra_precision=False, reference_system="ICRF")
    .to_pandas()
    .loc[:, ["epochJ2000secondsTDB", "RA", "DEC"]]
).reset_index(drop=True)
radec_horizons[["RA", "DEC"]] = radec_horizons[["RA", "DEC"]].apply(np.radians)
print(radec_mpc.head())
print(radec_horizons.head())

# print(radec_mpc.head())
# print("\n\n")
# print(radec_horizons.head())

diff = radec_horizons - radec_mpc
diff[["RA_deg", "DEC_deg"]] = diff[["RA", "DEC"]].apply(np.degrees)

print("max diff:")
print(diff.max())
print(diff)

print(type(radec_horizons))

   epochJ2000secondsTDB        RA       DEC
0          6.897485e+08  5.085946 -0.336130
1          6.897487e+08  5.085967 -0.336125
2          6.897491e+08  5.086008 -0.336113
3          6.897493e+08  5.086029 -0.336110
4          6.898301e+08  5.094551 -0.334148
   epochJ2000secondsTDB        RA       DEC
0          6.897485e+08  5.085946 -0.336129
1          6.897487e+08  5.085966 -0.336125
2          6.897491e+08  5.086008 -0.336115
3          6.897493e+08  5.086029 -0.336110
4          6.898301e+08  5.094550 -0.334148
max diff:
epochJ2000secondsTDB    3.982782e-04
RA                      1.047198e-06
DEC                     7.369168e-07
RA_deg                  6.000000e-05
DEC_deg                 4.222222e-05
dtype: float64
    epochJ2000secondsTDB            RA           DEC    RA_deg   DEC_deg
0               0.000396 -3.490659e-07  2.714957e-07 -0.000020  0.000016
1               0.000378 -1.163553e-07  1.357478e-07 -0.000007  0.000008
2               0.000383 -2.036217e-07 -1.9

In [7]:
a = eros.ephemerides(extra_precision=False, reference_system="ICRF")

print(type(a))

<class 'astropy.table.table.Table'>


In [12]:
radec_new = eros.interpolated_observations(degrees=True)
radec_mpc2 = radec_mpc.to_numpy()*180/np.pi

diff = (radec_new-radec_mpc2)

print(diff.max(axis=0))

# a = (
#     a.to_pandas()
#     .loc[:, ["epochJ2000secondsTDB", "RA", "DEC"]]
# ).reset_index(drop=True)
# # a[["RA", "DEC"]] = a[["RA", "DEC"]].apply(np.radians)

# print(a)
# print(batch.observatories)
# print(a.columns)

[-3.88299315e+10  6.39540000e-05  4.50902222e-05]


In [9]:
print()